Junior Data Science Klub - 2025-ös verseny
Tóth Zoltán

Az adatok előkészítése a nagy mennyiségre tekintettel SQL-ben történt (az adatokat a megadott csv-ből olvastam be). 
Ennek kódja (tábla létrehozűsa és utána a releváns események lekérése - azon userek összes esménye, ahol volt 'trial_started_event'): 

CREATE TABLE telepromter_fixed (
    event VARCHAR(255) NOT NULL,        
    event_time BIGINT NOT NULL,          
    distinct_id VARCHAR(255) NOT NULL, 
    os_version VARCHAR(50),            
    country_code VARCHAR(10)           

BULK INSERT dbo.telepromter_fixed
FROM 'C:\Users\tothz\Documents\2_bgf\jds\klubverseny_25\teleprompter_hashed_fixed_clean.csv'
WITH (
    FIELDTERMINATOR = ';',  
    ROWTERMINATOR = '\n',   
    FIRSTROW = 2            
);

select COUNT (event) from dbo.telepromter_fixed;

SELECT *
FROM dbo.telepromter_fixed
WHERE (distinct_id) IN (
    SELECT DISTINCT (distinct_id) as 'user'
    FROM dbo.telepromter_fixed
    WHERE event = 'trial_started_event'
);

SELECT *
INTO dbo.esemenyek
FROM dbo.telepromter_fixed
WHERE distinct_id IN (
    SELECT DISTINCT distinct_id
    FROM dbo.telepromter_fixed
    WHERE event = 'trial_started_event'
);



A további feldolgozás, elemzés Pythonban

In [4]:
import os
import pandas as pd
import numpy as np 

base_dir = r'C:\Users\tothz\Documents\2_bgf\jds\klubverseny_25'

In [5]:
df_trial_user = pd.read_csv(r'C:\Users\tothz\Documents\2_bgf\jds\klubverseny_25\trialosok_esemenyei_fixed.csv', sep=';', header=None, names=['event', 'event_time', 'distinct_id', 'os_version', 'country_code'])

In [8]:
df_trial_user['event_time_new'] = pd.to_datetime(df_trial_user['event_time'], unit='s')
#uj oszlop a most trailt kezdő userek közül konvertáltra
converted_users = df_trial_user[df_trial_user['event'].isin(['trial_converted_event', 'initial_purchase_event'])]['distinct_id'].unique()
df_trial_user['converted'] = df_trial_user['distinct_id'].isin(converted_users).astype(int)

df_trial_user.to_csv('teleprompter_1_fixed.csv', index=False)

#az egyes események száma 
esemenyek_csop = df_trial_user.groupby('event').count().reset_index()
esemenyek_csop.to_excel('esem_csop_fixed.xlsx', index=False)

#események szama converted-nem converted bontasban
esem_csop = df_trial_user.groupby(['event', 'converted']).size().reset_index(name='event_count')
esem_csop = esem_csop.pivot(index='event', columns='converted', values='event_count').fillna(0).astype(int)
esem_csop.columns = ['converted_0', 'converted_1']
esem_csop['sum'] = esem_csop['converted_0'] + esem_csop['converted_1']
esem_csop.to_csv('esem_csop.csv', index=False)

#szűkitünk arra, ahol a trail már befejeződött (7 nap)
condition_1 = (df_trial_user['event'] == 'trial_start_event') & (df_trial_user['event_time_new'] < '2025-02-22')
condition_2 = df_trial_user['event'].isin(['trial_converted_event', 'trial_cancelled_event', 'initial_purchase_event'])

# Azoknak a user_id-knek a kiválasztása, akiknél legalább az egyik feltétel teljesül
target_user_ids = df_trial_user[condition_1 | condition_2]['distinct_id'].unique()

# Szűrés azokra a sorokra, ahol a user_id szerepel a target_user_ids között - ez lesz az elemzo adatbazisunk
df_trial_2 = df_trial_user[df_trial_user['distinct_id'].isin(target_user_ids)]
df_trial_2.to_csv('esem_szuk_fixed.csv', index=False)


In [9]:
## ESEMENYTIPUSOK SZURESE
# csinálunk egy olyan táblát, amibe az egyes eseménytipusok vannak osszeszamolva convertalt, nem convertalt bontasban 

In [10]:
esem_csop = df_trial_2.groupby(['event', 'converted']).size().reset_index(name='event_count')
esem_csop = esem_csop.pivot(index='event', columns='converted', values='event_count').fillna(0).astype(int)
esem_csop.columns = ['converted_0', 'converted_1']
esem_csop['sum'] = esem_csop['converted_0'] + esem_csop['converted_1']
esem_csop.to_excel('esem_csop.xlsx', index=True)

In [11]:
#beolvassuk az esemnytipusok leirarsat es osszekotjkuk  a ket tablat

In [ ]:
esem_def=pd.read_excel(r'C:\Users\tothz\Documents\2_bgf\jds\klubverseny_25\TP_event_descriptions.xlsx')
esem_def=esem_csop.merge(sem_def, how='left', left_on='event', right_on='event_name')
esem_def.to_excel('esem_csop_2.xlsx', index=True)

In [ ]:
### Excelben készült egy funkciókat összevonó excel (esemény - eseménycsoport/funkció) a hibákat különákülön csoportkélnt kezelve az egyes funkcióknál
# ezt olvassukl be

In [ ]:
esem_csop=pd.read_excel(r'C:\Users\tothz\Documents\2_bgf\jds\klubverseny_25\esem_csop_3.xlsx', sheet_name='csoportok2')
df_trial_3=pd.read_csv('esem_szuk_fixed.csv')

In [ ]:
## Kiakalitjuk az osztályozáshoz az alaptáblát. Egy sor egy user, oszlopok az egyes eseménycsoportkban a user eseményei száma
#+ hozzárakjuk az egyéb user jellemző változokat

In [ ]:
df_trial_4 = df_trial_3.merge(esem_csop[['event_name', 'funkcio']], how='left', left_on='event', right_on='event_name')

# Felössszegzés az egyes 'distinct_id' és 'funkcio' kombinációkra
df_trial_5 = df_trial_4.groupby(['distinct_id', 'funkcio'])['funkcio'].count().reset_index(name='count')

# Pivotálás az események összesítéséhez 'distinct_id' szintjén
df_pivot = df_trial_5.pivot(index='distinct_id', columns='funkcio', values='count').fillna(0).astype(int)

# Az eredeti 'distinct_id'-nél szükséges user jellemzők (oszlopok) megőrzése
constant_columns = ['distinct_id', 'os_version', 'country_code', 'converted']
df_constants = df_trial_4[constant_columns].drop_duplicates(subset='distinct_id').set_index('distinct_id')

# Összefűzzük a pivotált DataFrame-et és a user tulajdonságokat 
df_alap = pd.concat([df_constants, df_pivot], axis=1).reset_index()

# A 'distinct_id' visszaállítása oszlopként
df_alap = df_alap.reset_index()
df_alap['distinct_id']  = df_alap['index']
df_alap = df_alap.drop(columns=['level_0', 'index'])

# oszlopsorrend megvaltoztatasa
df_alap = df_alap[['distinct_id', 'os_version', 'country_code', 'converted', 'account','login', 'payment','AI Script Generator',
       'AI Script Generator_error', 'Add Intro/Outro', 'Add Subtitles',
       'Add Subtitles_error', 'Add Watermark', 'Apple Watch_error',
       'Bluetooth Remote', 'Blur Background', 'Change Margins', 'Clean Audio',
       'Clean Audio_error', 'Countdown Before Starting', 'Custom Fonts',
       'Display Remaining Time', 'Export Raw Video', 'Export Raw Video_error',
       'Facebook', 'Flip Camera During Recording', 'Folder',
       'Font Size Change', 'Footpedal', 'Game Controller',
       'Keyboard Shortcuts', 'Lexend Support', 'Mirroring',
       'Open Dyslexic Support', 'Presentation Remote', 'Progress Bar',
       'Referral Program', 'Registration', 'Registration_error',
       'Replace Background', 'Replace Green Screen', 'Resize Video',
       'Resize Video_error', 'Script_generation', 'Script_generation_error',
       'Select Camera', 'Select FPS', 'Select Microphone',
       'Select Microphone_error', 'Social Log In', 'Speed Based Scrolling',
       'Stream with URL', 'Tap to Scroll', 'Trim Video', 'Visual Timer',
       'Web-based Remote', 'YouTube', 'account_error', 'facebook',
       'live_stream', 'live_stream_error', 'login_error',
       'onboarding', 'onboarding_error', 'other_error', 'payment_error', 'recording', 'recording_error', 'remote',
       'remote_error', 'select camera']]



In [ ]:
# időváltozók képzése és hozáfűzése

time_features = df_trial_4.groupby('distinct_id')['event_time_new'].agg(['min', 'max']).reset_index()
df_alap = pd.merge(df_alap, time_features, on='distinct_id', how='left')

In [ ]:
## DF_ALAP lesz az alaptáblánk a továbi elemzéshez, ahol funkciókat és nem eseményket használunk 

In [ ]:
### OSZTÁLYOZÁS - konverted a célváltozó (1)

In [ ]:
# első modell: logisztikus regresszió

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



X = df_alap.drop(columns=['converted', 'distinct_id', 'os_version', 'country_code', 'min', 'max'])
y = df_alap['converted']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

#kiértékelés (confuzios matrix, auroc)

y_pred = model.predict(X_test)
print("Pontosság:", accuracy_score(y_test, y_pred))
print("Jelentés:\n", classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot(cmap='Blues')

y_pred_proba = model.predict_proba(X_test)[:, 1]

auroc = roc_auc_score(y_test, y_pred_proba)
print(f"AUROC: {auroc:.3f}")


In [ ]:
## második model: döntési fa, random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

#NaN kezelése az os versionban
df_alap['os_version'] = df_alap['os_version'].fillna('Unknown')

# countrz code és os version átkodolása
le_country = LabelEncoder()
df_alap['country_code_encoded'] = le_country.fit_transform(df_alap['country_code'])

le_os_version = LabelEncoder()
df_alap['os_version_encoded'] = le_os_version.fit_transform(df_alap['os_version'])


# Az input változók kiválasztása 
X2 = df_alap.drop(columns=['distinct_id', 'converted', 'country_code','os_version', 'min', 'max'])
y2 = df_alap['converted']

# Adatok felosztása

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.3, random_state=42)


# Modell tanitas
rf_model2 = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model2.fit(X_train2, y_train2)

# Modell értékelése (konfuzios matrix, AUROC)
y_pred2 = rf_model2.predict(X_test2)

print("Confusion Matrix:")
print(confusion_matrix(y_test2, y_pred2))
print("\nClassification Report:")
print(classification_report(y_test2, y_pred2))

y_pred_proba2 = rf_model2.predict_proba(X_test2)[:, 1]
auroc = roc_auc_score(y_test2, y_pred_proba2)
print(f"AUROC: {auroc:.3f}")

# Feature importance
importances = rf_model2.feature_importances_
feature_names = X2.columns
importance_df2 = pd.DataFrame({'Feature': feature_names, 'Importance': importances}).sort_values(by='Importance', ascending=False)
print("\nFeature Importance:")
print(importance_df2)


In [ ]:
## harmadik modell: gradient boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

## input és célváltozók azonosak, mint az előbb, teszt-tény szétválasztás is ugyanaz - indul a tanitás

gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_model.fit(X_train2, y_train2)

# Modell értékelése (konfúzios matrix, AUROC)
y_pred3 = gb_model.predict(X_test2)

print("Confusion Matrix:")
print(confusion_matrix(y_test2, y_pred3))
print("\nClassification Report:")
print(classification_report(y_test2, y_pred3))

y_pred_proba3 = gb_model.predict_proba(X_test2)[:, 1]
auroc2 = roc_auc_score(y_test2, y_pred_proba3)
print(f"AUROC: {auroc:.3f}")

# Feature importance
importances3 = gb_model.feature_importances_
feature_names = X2.columns
importance_df3 = pd.DataFrame({'Feature': feature_names, 'Importance': importances}).sort_values(by='Importance', ascending=False)
print("\nFeature Importance:")
print(importance_df3)

In [ ]:
## a random forest és gradient boosting  akét hazsnálható model, GB jobb
# ezeknél a feature imnportance kimentése

In [ ]:
importance_df2.to_excel('feature_importance2.xlsx', index=False)
importance_df3.to_excel('feature_importance3.xlsx', index=False)
df_alap.to_excel('df_alap.xlsx', index=False)

In [ ]:
# ROC görbe megjelenítése

fpr1, tpr1, _ = roc_curve(y_test2, y_pred_proba2)
fpr2, tpr2, _ = roc_curve(y_test2, y_pred_proba3)

plt.figure(figsize=(8, 6))
plt.plot(fpr1, tpr1, label=f"Model 1 (AUROC = {auroc:.3f})")
plt.plot(fpr2, tpr2, label=f"Model 2 (AUROC = {auroc2:.3f})")
plt.plot([0, 1], [0, 1], 'k--', label="Random Guess")

plt.xlabel("False Positive Rate (FPR)")
plt.ylabel("True Positive Rate (TPR)")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.legend(loc="lower right")
plt.grid()
plt.show()

In [ ]:
### KLASZTEREZES - KMEAN algoritmus

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt 

#adatok kiválasztása
df_alap_cluster= df_alap[['converted', 'account','login', 'payment','AI Script Generator',
       'AI Script Generator_error', 'Add Intro/Outro', 'Add Subtitles',
       'Add Subtitles_error', 'Add Watermark', 'Apple Watch_error',
       'Bluetooth Remote', 'Blur Background', 'Change Margins', 'Clean Audio',
       'Clean Audio_error', 'Countdown Before Starting', 'Custom Fonts',
       'Display Remaining Time', 'Export Raw Video', 'Export Raw Video_error',
       'Facebook', 'Flip Camera During Recording', 'Folder',
       'Font Size Change', 'Footpedal', 'Game Controller',
       'Keyboard Shortcuts', 'Lexend Support', 'Mirroring',
       'Open Dyslexic Support', 'Presentation Remote', 'Progress Bar',
       'Referral Program', 'Registration', 'Registration_error',
       'Replace Background', 'Replace Green Screen', 'Resize Video',
       'Resize Video_error', 'Script_generation', 'Script_generation_error',
       'Select Camera', 'Select FPS', 'Select Microphone',
       'Select Microphone_error', 'Social Log In', 'Speed Based Scrolling',
       'Stream with URL', 'Tap to Scroll', 'Trim Video', 'Visual Timer',
       'Web-based Remote', 'YouTube', 'account_error', 'facebook',
       'live_stream', 'live_stream_error', 'login_error',
       'onboarding', 'onboarding_error', 'other_error', 'payment_error', 'recording', 'recording_error', 'remote',
       'remote_error', 'select camera','country_code_encoded',
       'os_version_encoded']]

# Adatok skálázása szukseges - standard scalerrel
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_alap_cluster)



In [ ]:
# Ideális klaszterszám kiválasztása (elbow modszerrel, grafikusan)
inertia = []
range_clusters = range(1, 30)
for k in range_clusters:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(scaled_data)
    inertia.append(kmeans.inertia_)

# Elbow görbe plot
plt.figure(figsize=(8, 6))
plt.plot(range_clusters, inertia, marker='o')
plt.xlabel("Klaszterek száma")
plt.ylabel("Inertia")
plt.title("Elbow Method az optimális klaszterszámhoz")
plt.grid()
plt.show()

In [ ]:
## 5-os klaszeterszamra klszeterzés - Kmeans
kmeans = KMeans(n_clusters=5, random_state=42)  
kmeans.fit(df_alap_cluster)

# Klaszter címkék hozzárendelése az eredeti DataFrame-hez
df_alap['Cluster'] = kmeans.labels_


In [ ]:
# Klaszterekben a konvertált arany 
cluster_analysis = df_alap.groupby('Cluster').agg(total=('converted', 'size'), converted_mean=('converted', 'mean')).reset_index()

print("Klaszterek összegzése:")
print(cluster_analysis)

In [ ]:
# Klaszterek vizualizációja PCA segítségével (2D térben)
pca = PCA(n_components=2)
pca_data = pca.fit_transform(df_alap_cluster)

plt.figure(figsize=(8, 6))
plt.scatter(pca_data[:, 0], pca_data[:, 1], c=df_alap['Cluster'], cmap='viridis', s=50)
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.title("K-means klaszterek vizualizációja (PCA)")
plt.colorbar(label='Cluster')
plt.grid()
plt.show()


In [ ]:
## load matrix kiexportalása
print("PCA komponens súlyai (load matrix):")
print(pca.components_)

# PCA komponensek súlyainak DataFrame-be rendezése
components_df = pd.DataFrame(pca.components_, columns=df_alap_cluster.columns, index=[f"PC{i+1}" for i in range(pca.n_components_)])

components_df.to_excel('pca_load_matrix.xlsx', index=True)


In [ ]:
## KLASZTEREK további elemzése

In [ ]:
#relativ gyakoriságok
cluster_summary1 = cluster_analysis.groupby('Cluster').mean()

cluster_summary2 = df_alap.groupby('Cluster').mean()

cluster_summary2.to_excel('cluster_summary2.xlsx', index=False)
cluster_summary1.to_excel('cluster_summary1.xlsx', index=False)

In [ ]:
##### A KOVETKEZOKBEN AZ ESEMÉNYEKET ÖSSZEGEZZÜK FEL A USER SZINTJÉN, NEM AZ ESEMÉNYCSOPORTOKAT - TÖBB VÁLTOZÓ 

In [ ]:
## kinduló adatvbázis: user-re grupba, de nem funkció, hanem esemény
# Számoljuk meg az egyes 'distinct_id' és 'event_name' kombinációkat
df_trial_esem = df_trial_4.groupby(['distinct_id', 'event_name'])['event_name'].count().reset_index(name='count')

# Pivotálás az események összesítéséhez 'distinct_id' szintjén
df_pivot_esem = df_trial_esem.pivot(index='distinct_id', columns='event_name', values='count').fillna(0).astype(int)

# Az eredeti 'distinct_id'-nél a user tulajdonságok megőrzése - ez már fent ekészlt a funkcióra pivotáláskor
# Összefűzzük a pivotált DataFrame-et és a tulajdonságokat
df_alap_esem = pd.concat([df_constants, df_pivot_esem], axis=1).reset_index()

# A 'distinct_id' visszaállítása oszlopként
df_alap_esem = df_alap_esem.reset_index()
df_alap_esem['distinct_id']  = df_alap_esem['index']
df_alap_esem = df_alap_esem.drop(columns=['level_0', 'index'])

In [ ]:
### NEM FOGUNK klaszterezni, de az osztályozást megcsináljuk - a két jobb modellel

In [ ]:
#random forest

In [ ]:
#NaN kezelése az os versionban
df_alap_esem['os_version'] = df_alap_esem['os_version'].fillna('Unknown')

# szukseges átkodolas
le_country = LabelEncoder()
df_alap_esem['country_code_encoded'] = le_country.fit_transform(df_alap_esem['country_code'])

le_os_version = LabelEncoder()
df_alap_esem['os_version_encoded'] = le_os_version.fit_transform(df_alap_esem['os_version'])


# Az input változók kiválasztása
Xe2 = df_alap_esem.drop(columns=['distinct_id', 'converted', 'country_code','os_version', 'billing_issue_event', 
    'cancellation_event', 'expiration_event', 'initial_purchase_event' , 'product_change_event', 'renewal_event',
    'trial_cancelled_event' , 'trial_converted_event', 'trial_started_event' , 'uncancellation_event', 'subscriptionStatus_didChange'])  #itt meg a vconvertalasi esemenyeket is ki kell venni
ye2 = df_alap_esem['converted']

# Adatok felosztása

X_train_e2, X_test_e2, y_train_e2, y_test_e2 = train_test_split(Xe2, ye2, test_size=0.3, random_state=42)

# tanitas
rf_model_e2 = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_e2.fit(X_train_e2, y_train_e2)

# kiértékelés - konfizioos matrix, auroc 
y_pred_e2 = rf_model_e2.predict(X_test_e2)

print("Confusion Matrix:")
print(confusion_matrix(y_test_e2, y_pred_e2))
print("\nClassification Report:")
print(classification_report(y_test_e2, y_pred_e2))

y_pred_proba_e2 = rf_model_e2.predict_proba(X_test_e2)[:, 1]
auroc_e2 = roc_auc_score(y_test_e2, y_pred_proba_e2)
print(f"AUROC: {auroc_e2:.3f}")

# Feature importance
importances = rf_model_e2.feature_importances_
feature_names = Xe2.columns
importance_df_e2 = pd.DataFrame({'Feature': feature_names, 'Importance': importances}).sort_values(by='Importance', ascending=False)
print("\nFeature Importance:")
print(importance_df_e2)

In [ ]:
##Gradient boosting

In [ ]:
X_train_e2, X_test_e2, y_train_e2, y_test_e2 = train_test_split(Xe2, ye2, test_size=0.3, random_state=42)

#tanitas
gb_model_e3 = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_model_e3.fit(X_train_e2, y_train_e2)

# kiértékelés
y_pred_e3 = gb_model_e3.predict(X_test_e2)

print("Confusion Matrix:")
print(confusion_matrix(y_test_e2, y_pred_e3))
print("\nClassification Report:")
print(classification_report(y_test_e2, y_pred_e3))

y_pred_proba_e3 = gb_model_e3.predict_proba(X_test_e2)[:, 1]
auroc_e3 = roc_auc_score(y_test_e2, y_pred_proba_e3)
print(f"AUROC: {auroc_e3:.3f}")

# Feature importance
importances_e3 = gb_model_e3.feature_importances_
feature_names = Xe2.columns
importance_dfe3 = pd.DataFrame({'Feature': feature_names, 'Importance': importances}).sort_values(by='Importance', ascending=False)
print("\nFeature Importance:")
print(importance_dfe3)

In [ ]:
## A legjobban teljesitő modell - gradient boosting - fejlesztése

In [ ]:
# False negati arány csökentése súlyozással 

In [ ]:
# Gradient Boosting létrehozása osztálysúlyokkal
gb_model_e3FN = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)

# Fit minták egyedi súlyozásával
from sklearn.utils.class_weight import compute_sample_weight

# Példák súlyainak számítása az y_train osztályainak figyelembevételével
sample_weights = compute_sample_weight(class_weight='balanced', y=y_train_e2)

# Modell betanítása
gb_model_e3FN.fit(X_train_e2, y_train_e2, sample_weight=sample_weights)

## kiértékelés
y_pred_e3FN = gb_model_e3FN.predict(X_test_e2)

print("Confusion Matrix:")
print(confusion_matrix(y_test_e2, y_pred_e3FN))
print("\nClassification Report:")
print(classification_report(y_test_e2, y_pred_e3FN))

y_pred_proba_e3FN = gb_model_e3FN.predict_proba(X_test_e2)[:, 1]
auroc_e3FN = roc_auc_score(y_test_e2, y_pred_proba_e3FN)
print(f"AUROC: {auroc_e3FN:.3f}")

importances_e3FN = gb_model_e3.feature_importances_
feature_names = Xe2.columns
importance_dfe3FN = pd.DataFrame({'Feature': feature_names, 'Importance': importances}).sort_values(by='Importance', ascending=False)
print("\nFeature Importance:")
print(importance_dfe3FN)


In [ ]:
## Oversampling 

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

# A korábbi teszt-train felosztást és változó kiválasztást használhatjuk 
# SMOTE alkalmazása a train adatokra
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_e2, y_train_e2)

# model betanítása az uj adatokon
gb_model_e3o = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_model_e3o.fit(X_train_resampled, y_train_resampled)

# kiértékelés
y_pred_e3o = gb_model_e3o.predict(X_test_e2)

print("Confusion Matrix:")
print(confusion_matrix(y_test_e2, y_pred_e3o))
print("\nClassification Report:")
print(classification_report(y_test_e2, y_pred_e3o))

y_pred_proba_e3o = gb_model_e3o.predict_proba(X_test_e2)[:, 1]
auroc_e3o = roc_auc_score(y_test_e2, y_pred_proba_e3o)
print(f"AUROC: {auroc_e3o:.3f}")

In [ ]:
### SMOTE és class weight együttese

In [ ]:
# SMOTE alkalmazása a train adatokra
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_e2, y_train_e2)

# sample weights ujraszamolasa
sample_weights3 = compute_sample_weight(class_weight='balanced', y=y_train_resampled)


# model betanítása
gb_model_komb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_model_komb.fit(X_train_resampled, y_train_resampled, sample_weight=sample_weights3)

# kiértékelése
y_pred_komb = gb_model_komb.predict(X_test_e2)

print("Confusion Matrix:")
print(confusion_matrix(y_test_e2, y_pred_komb))
print("\nClassification Report:")
print(classification_report(y_test_e2, y_pred_komb))

y_pred_proba_komb = gb_model_komb.predict_proba(X_test_e2)[:, 1]
auroc_komb = roc_auc_score(y_test_e2, y_pred_proba_komb)
print(f"AUROC: {auroc_komb:.3f}")

In [ ]:
#### A class weight érdemben javitotta a Recall-t, kis AUROC és accuracy veszteség mellett 
# az oversampling gyengébb volt
# a kombinált modell a javiott sólyozással zámoló modelt már nem javitotta érdemben, ezért elvetem: marad BG + sulyozas 

In [ ]:
## Hozzáfúzzük aze redeti  df-hez uj oszlpként a legjobb modellel kiszámitott 1 osztályba esési valósziműséget
df_alap_esem_2 = df_alap_esem

# Az oszlopokat kiválasztjuk a teljes adatbázisból (df_alap_esem), amit a modell megért.
X_full = df_alap_esem_2[X_train_e2.columns]  # Biztosítsuk, hogy az X_full azonos struktúrájú legyen, mint a tanítókészlet.

# 1-es osztály valószínűség kiszámítása a teljes adatbázisra.
df_alap_esem_2['class_1_probability'] = gb_model_e3FN.predict_proba(X_full)[:, 1]


In [ ]:
## clustert még hozza kell tenni a df_alap -ból, a distinct ID-k alapján 

In [ ]:
df_cel = df_alap_esem_2.merge(df_alap[['distinct_id', 'Cluster']], how='left', on='distinct_id')
df_cel.to_excel('cel.xlsx', index= 'False')

In [ ]:
esem_csop = esem_csop.set_index('event_name')
importance_dfe3FN = importance_dfe3FN.set_index('Feature').join(esem_csop[['sum', 'converted_0', 'converted_1']],how='left').reset_index()
importance_dfe3FN.to_excel('feature_importnace_FN_gyakorisagok.xlsx', index= 'False')

In [ ]:
### UJ ELEMZES - csak a trial period alatti események alapjan osztályozás, konvertálás prediktálása (konverzio valószinűsége kiszámitása)

In [ ]:
# alaptábla az eseményket még nem aggregáltan tartalmazó df_trial_4 tábla - ezt kell leszűrnünk a kivánt eseményekre

from datetime import timedelta

# 1. Az első 'trial start event' időpontjának azonosítása userenként
df_trial_4['event_time_new'] = pd.to_datetime(df_trial_4['event_time_new'])  # Ha még nem datetime
first_trial_times = df_trial_4[df_trial_4['event'] == 'trial_started_event'] \
    .groupby('distinct_id')['event_time_new'].min().reset_index()
first_trial_times.rename(columns={'event_time_new': 'first_trial_time'}, inplace=True)

df_trial_4 = df_trial_4.drop(columns=['first_trial_time'], errors='ignore')
df_trial_szuk = df_trial_4.merge(first_trial_times, on='distinct_id', how='left')

# Időkülönbség számítása az első eseményhez képest
df_trial_szuk['time_diff'] = df_trial_szuk['event_time_new'] - df_trial_szuk['first_trial_time']

# Szűrés a feltételek alapján
df_trial_szuk = df_trial_szuk[(df_trial_szuk['time_diff'] <= timedelta(days=7)) | 
    (df_trial_szuk['event'] == 'trial_converted_event')].copy()



In [ ]:
## kinduló adatvbázis: user-re grupba, de nem funkció, hanem esemény, és csak a szukitet adatokat (trial strat + 7 nap)
# Számoljuk meg az egyes 'distinct_id' és 'event_name' kombinációkat
df_trial_esem_szuk = df_trial_szuk.groupby(['distinct_id', 'event_name'])['event_name'].count().reset_index(name='count')

# Pivotálás az események összesítéséhez 'distinct_id' szintjén
df_pivot_esem_szuk = df_trial_esem_szuk.pivot(index='distinct_id', columns='event_name', values='count').fillna(0).astype(int)

# Az eredeti 'distinct_id'-nél állandó user tulajdonsagok megőrzése - ez már fent ekészlt  afunkcióra pivotáláskor

# Összefűzzük a pivotált DataFrame-et és a tulajdonsagokat
df_alap_esem_szuk = pd.concat([df_constants, df_pivot_esem_szuk], axis=1).reset_index()

# A 'distinct_id' visszaállítása oszlopként
df_alap_esem_szuk = df_alap_esem_szuk.reset_index()
df_alap_esem_szuk['distinct_id']  = df_alap_esem_szuk['index']
df_alap_esem_szuk = df_alap_esem_szuk.drop(columns=['level_0', 'index'])



In [ ]:
### OSZTALYOZAS

In [ ]:
### gradient boosting alap

In [ ]:
#NaN kezelése az os versionban
df_alap_esem_szuk['os_version'] = df_alap_esem_szuk['os_version'].fillna('Unknown')

# szukseges atkodolas
le_country = LabelEncoder()
df_alap_esem_szuk['country_code_encoded'] = le_country.fit_transform(df_alap_esem_szuk['country_code'])

le_os_version = LabelEncoder()
df_alap_esem_szuk['os_version_encoded'] = le_os_version.fit_transform(df_alap_esem_szuk['os_version'])


# Az input változók kiválasztása
Xs2 = df_alap_esem_szuk.drop(columns=['distinct_id', 'converted', 'country_code','os_version',  
    'cancellation_event', 'expiration_event', 'initial_purchase_event' , 'product_change_event', 'renewal_event',
    'trial_cancelled_event' , 'trial_converted_event', 'trial_started_event' , 'uncancellation_event', 'subscriptionStatus_didChange'])  #itt meg a vconvertalasi esemenyeket is ki kell venni
ys2 = df_alap_esem_szuk['converted']

# Adatok felosztása

X_train_s2, X_test_s2, y_train_s2, y_test_s2 = train_test_split(Xs2, ys2, test_size=0.3, random_state=42)

#tanitas
gb_model_s3 = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_model_s3.fit(X_train_s2, y_train_s2)

# kiértékelés
y_pred_s3 = gb_model_s3.predict(X_test_s2)

print("Confusion Matrix:")
print(confusion_matrix(y_test_s2, y_pred_s3))
print("\nClassification Report:")
print(classification_report(y_test_s2, y_pred_s3))

y_pred_proba_s3 = gb_model_s3.predict_proba(X_test_s2)[:, 1]
auroc_s3 = roc_auc_score(y_test_s2, y_pred_proba_s3)
print(f"AUROC: {auroc_s3:.3f}")


In [ ]:
### False Negativok aránya javitésa sőlyozássaé 

In [ ]:
# Gradient Boosting létrehozása osztálysúlyokkal
gb_model_s3FN = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)

# Fit minták egyedi súlyozásával
from sklearn.utils.class_weight import compute_sample_weight

# Példák súlyainak számítása az y_train osztályainak figyelembevételével
sample_weights = compute_sample_weight(class_weight='balanced', y=y_train_s2)

# Modell betanítása
gb_model_s3FN.fit(X_train_s2, y_train_s2, sample_weight=sample_weights)

# FN Modell kiértékelése
y_pred_s3FN = gb_model_s3FN.predict(X_test_s2)

print("Confusion Matrix:")
print(confusion_matrix(y_test_s2, y_pred_s3FN))
print("\nClassification Report:")
print(classification_report(y_test_s2, y_pred_s3FN))

y_pred_proba_s3FN = gb_model_s3FN.predict_proba(X_test_s2)[:, 1]
auroc_s3FN = roc_auc_score(y_test_s2, y_pred_proba_s3FN)
print(f"AUROC: {auroc_s3FN:.3f}")

In [ ]:
# SMOTE + FN

In [ ]:
# SMOTE alkalmazása a train adatokra
smote = SMOTE(random_state=42)
X_train_resampled_s, y_train_resampled_s = smote.fit_resample(X_train_s2, y_train_s2)

# sample weights ujraszamolasa
sample_weights3_s = compute_sample_weight(class_weight='balanced', y=y_train_resampled_s)


# model betanítása
gb_model_komb_s = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_model_komb_s.fit(X_train_resampled_s, y_train_resampled_s, sample_weight=sample_weights3_s)

# kiértékelés
y_pred_komb_s = gb_model_komb_s.predict(X_test_s2)

print("Confusion Matrix:")
print(confusion_matrix(y_test_s2, y_pred_komb_s))
print("\nClassification Report:")
print(classification_report(y_test_s2, y_pred_komb_s))

y_pred_proba_komb_s = gb_model_komb_s.predict_proba(X_test_s2)[:, 1]
auroc_komb_s = roc_auc_score(y_test_s2, y_pred_proba_komb_s)
print(f"AUROC: {auroc_komb_s:.3f}")


In [ ]:
### legjobb a sulyozassal készült model

In [ ]:
# Feature importance - s FN
importances_s3FN = gb_model_s3FN.feature_importances_
feature_names = Xs2.columns
importance_dfs3FN = pd.DataFrame({'Feature': feature_names, 'Importance': importances_s3FN}).sort_values(by='Importance', ascending=False)

importance_dfs3FN.to_excel('f_imp_s3FN.xlsx')

# Feature importance s kombi
importances_komb_s = gb_model_komb_s.feature_importances_
feature_names = Xs2.columns
importance_df_komb_s = pd.DataFrame({'Feature': feature_names, 'Importance': importances_komb_s}).sort_values(by='Importance', ascending=False)

importance_df_komb_s.to_excel('f_imp_komb_s.xlsx')

In [ ]:
### legjobb model - sfn valószinúségei hozzáfúzése a userekhez

In [ ]:
## Hozzáfúzzük aze redeti  df-hez uj oszlpként a legjobb modellel kiszámitott 1 osztályba esési valósziműséget
df_alap_esem_szuk2 = df_alap_esem_szuk

# Az oszlopokat kiválasztjuk a teljes adatbázisból (df_alap_esem), amit a modell megért.
X_full = df_alap_esem_szuk2[X_train_s2.columns]  # Biztosítsuk, hogy az X_full azonos struktúrájú legyen, mint a tanítókészlet.

# 1-es osztály valószínűség kiszámítása a teljes adatbázisra.
df_alap_esem_szuk2['class_1_probability'] = gb_model_s3FN.predict_proba(X_full)[:, 1]


In [ ]:
df_cel_szuk = df_alap_esem_szuk2.merge(df_alap[['distinct_id', 'Cluster']], how='left', on='distinct_id')

In [ ]:
df_alap_esem_szuk2.to_excel('df_alap_esem_szuk.xlsx', index = False)

In [ ]:
##### egyelőre vége :-) ####